In [ ]:
from fluvial_particle.RiverGrid import RiverGrid
from fluvial_particle.Particles import Particles
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from vtk.util import numpy_support
from IPython.display import Video

In [ ]:
# Set simulation parameters
output_directory = "../tests/test"
file_name_2d = "../tests/data/Result_FM_MEander_1_long_2D1.vtk"
file_name_3d = "../tests/data/Result_FM_MEander_1_long_3D1_new.vtk"
endtime = 1460.0
dt = 0.25
PrintAtTick = 10.0
track3d = 1
nparticles = 1000
xstart, ystart, zstart = (6.14, 9.09, 10.3)
lev = 0.00025

# Pre-compute simulation times
times = np.arange(dt, endtime + dt, dt)
n_times = times.size
print_inc = np.max([np.int32(PrintAtTick / dt), 1])  # bound below
print_inc = np.min([print_inc, n_times])  # bound above
print_times = times[print_inc - 1 : n_times : print_inc]
if print_times[-1] != times[-1]:
    # add final time to print_times if not already
    print_times = np.append(print_times, times[-1])
n_prints = print_times.size + 1  # plus one so we can write t=0 to file

In [ ]:
# Initialize river mesh
river = RiverGrid(track3d, file_name_2d, file_name_3d)

In [ ]:
# Initialize particles
rng = np.random.default_rng(seed=12345)
x = np.full((nparticles,), fill_value=xstart)
y = np.full((nparticles,), fill_value=ystart)
z = np.full((nparticles,), fill_value=zstart)
particles = Particles(nparticles, x, y, z, rng, river, Track3D=track3d, lev=lev)

In [ ]:
# Validate initial positions of the particles
particles.initial_validation(starttime=0.0)

In [ ]:
# Create output particles HDF5 file
fname = output_directory + "//particles.h5"
parts_h5 = particles.create_hdf5(n_prints, nparticles, fname=fname)
particles.write_hdf5(parts_h5, 0, 0, nparticles, 0.0, 0)  # write initial condition

In [ ]:
# Move the particles
for i in range(n_times):
    particles.move(times[i], dt)
    tidx = np.searchsorted(print_times, times[i])
    if print_times[tidx] == times[i]:
        # Write to file at specified times
        particles.write_hdf5(parts_h5, np.int32(i / print_inc) + 1, 0, nparticles, times[i], 0)
        print(f"Elapsed simulation time: {times[i]} seconds")

parts_h5.close()  # always close the HDF5 file!

In [ ]:
def _animate(h5fname, mvname, **ani_kwargs):
    h5f = h5py.File(h5fname, "r")
    grp = h5f["coordinates"]
    hx = grp["x"]
    hy = grp["y"]
    times = grp["time"]
    hh = h5f["properties"]["htabvbed"]

    fig, ax = plt.subplots(figsize=(10.,2.5), subplot_kw={"aspect":"equal"})
    gr = ax.scatter(xgrid, ygrid)
    sc = ax.scatter(hx[0,:], hy[0,:], s=2)

    def animate(i):
        ax.set_title(f"Simulation time: {times[i].item()}")
        sc.set_offsets(np.c_[hx[i,:], hy[i,:]])

    ani = matplotlib.animation.FuncAnimation(fig, animate, **ani_kwargs)

    plt.close()
    ani.save(mvname, dpi=100)
    h5f.close()

fname = output_directory + "//particles.h5"
mvname = output_directory + "//test.mp4"
ani_kwargs = {"interval": 100}

vtkcoords = river.vtksgrid2d.GetPoints().GetData()
coords = numpy_support.vtk_to_numpy(vtkcoords)
xgrid = coords[:, 0]
ygrid = coords[:, 1]

_animate(fname, mvname, **ani_kwargs)

Video(mvname, html_attributes="controls autoplay loop")
